#  Creating an aggregate view of household data

The purpose of this notebook is to create a dataframe of all data available on a houshold that has been aggregated to provide a rich view of the behavior and trends of each household for segmentation via clustering algorithm. 

In [8]:
import sqlite3
import pandas as pd
from pathlib import Path

In [9]:
# I've already created a db that can be called on in a prievious notebook
# This re-establishes the connection to 'trial.db'
Path('trial.db').touch()
con = sqlite3.connect('trial.db')
c = con.cursor()

## Exploring aggregate features for households focused on coupon redemption

The below are keys for data frames that will be concatenated together for a data frame of aggregate information about each household:


#### Coupon/Campaign Redemption information


df1 = # of campaigns redeemed

df2 = # of campaigns sent

df3 = % of campaigns redeemed

df4 = # of coupons redeemed

df5 = # of coupons sent

df6 = % of coupons redeemed

df7 = # of days on which a coupon was redeemed

df8 = # of TypeA campaigns sent

df9 = # of TypeB campaigns sent

df10 = # of TypeC campaigns sent

df11 = % of campaigns sent that were TypeA

df12 = % of campaigns sent that were TypeB

df13 = % of campaigns sent that were TypeC


#### Products purchased information


df14 = # of unique products purchased

df15 = # of departments purchased from

df16 = most frequent department purchased from

df17 = least frequent department purchased from

df18 = # of unique sub-category descriptions

df19 = most frequently purchased product

df20 = least frequently purchased product

df21 = product with the most volum purchased ##Currently not working properly##

df22 = Preferred Brand (National vs Private) ##Scrapped - same information as df23##

df23 = % of products purchased which are from a Private brand

df24 = % of products purchased from preferred department


#### Transaction information


df25 = Average basket sales value

df26 = Max basket sales value

df27 = Min basket sales value

df28 = total # of baskets

df29 = Average # of items in basket

df30 = Max # of items in basket

df31 = Min # of items in basket

df32 = Most frequent item in basket

df33 = Average Manufacturer discout

df34 = Max Manufacturer discout

df35 = Min Manufacturer discout

df36 = Average Coupon discount

df37 = Max Coupon discount

df38 = Min Coupon discount

df39 = Average Loyalty discount

df40 = Max Loyalty discount

df41 = Min Loyalty discount

df42 = # of stores purchased from

df43 = Most frequent store

df44 = Most frequent time of day to purchase

df45 = 1st day of purchase

df46 = last day of purchase

df47 = RECENCY (RFM analysis)

df48 = FREQUENCY (RFM analysis)

df549 = MONETARY (RFM analysis)

df50 = Avg # of trips per week

df51 = %% of purchases made where the product purchased was displayed in the store

df52 = %% of purchases made where the product purchased was displayed in the mailer

df53= %% of baskets included an item which was on display in the mailer

df54= %% of baskets included an item which was on display in the store

df55= Avg quantity of product purchaed

df56= Avg # of days in between shopping trips

In [10]:
# This cell is used to check queries to ensure I'm creating the correct dataframes below using pd.read_sql
c.execute("""
SELECT * FROM HH_DEMOGRAPHIC
""")
c.fetchall()

[('65+',
  'A',
  '35-49K',
  'Homeowner',
  '2 Adults No Kids',
  '2',
  'None/Unknown',
  1),
 ('45-54',
  'A',
  '50-74K',
  'Homeowner',
  '2 Adults No Kids',
  '2',
  'None/Unknown',
  7),
 ('25-34', 'U', '25-34K', 'Unknown', '2 Adults Kids', '3', '1', 8),
 ('25-34', 'U', '75-99K', 'Homeowner', '2 Adults Kids', '4', '2', 13),
 ('45-54',
  'B',
  '50-74K',
  'Homeowner',
  'Single Female',
  '1',
  'None/Unknown',
  16),
 ('65+',
  'B',
  'Under 15K',
  'Homeowner',
  '2 Adults No Kids',
  '2',
  'None/Unknown',
  17),
 ('45-54',
  'A',
  '100-124K',
  'Homeowner',
  '2 Adults No Kids',
  '2',
  'None/Unknown',
  18),
 ('35-44', 'B', '15-24K', 'Unknown', 'Single Female', '1', 'None/Unknown', 19),
 ('25-34',
  'A',
  '75-99K',
  'Renter',
  '2 Adults No Kids',
  '2',
  'None/Unknown',
  20),
 ('45-54',
  'A',
  '75-99K',
  'Homeowner',
  '2 Adults No Kids',
  '2',
  'None/Unknown',
  22),
 ('35-44', 'U', '50-74K', 'Unknown', 'Unknown', '1', 'None/Unknown', 25),
 ('45-54',
  'U',
  '

The below cells store each query aggregating information in dataframes to be combined later

In [11]:
# Creates a column which reflects the total number of campaigns a household redeemed. 
df1 = pd.read_sql("""
SELECT HOUSEHOLD_KEY, COUNT(DISTINCT CAMPAIGN) AS CampaignsRedeemed
FROM REDEEMED 
GROUP BY HOUSEHOLD_KEY""", con, index_col='household_key')

In [12]:
# Creates column which reflects the total number of campaigns sent to a household
df2 = pd.read_sql("""
SELECT HOUSEHOLD_KEY, COUNT(DISTINCT CAMPAIGN) AS CampaignsSent
FROM CAMPAIGN_TABLE
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [13]:
# Creates a column that shows the %% of campaigns which a household redeemed
df3 = pd.read_sql("""
SELECT R.HOUSEHOLD_KEY as household_key, (COUNT(DISTINCT R.CAMPAIGN)*1.0/COUNT(DISTINCT CT.CAMPAIGN)) 
AS Percent_CampaignRedeemed
FROM REDEEMED AS R
LEFT JOIN CAMPAIGN_TABLE AS CT 
ON R.HOUSEHOLD_KEY= CT.HOUSEHOLD_KEY
GROUP BY R.HOUSEHOLD_KEY""", con, index_col='household_key')

In [14]:
# Creates a colum that shows the count of coupons redeemed
df4 = pd.read_sql("""
SELECT HOUSEHOLD_KEY, COUNT(COUPON_UPC) AS CouponRedeemed_Count
FROM REDEEMED
GROUP BY HOUSEHOLD_KEY""", con, index_col='household_key')

In [15]:
# Creates a column totalling all coupons sent to a household
df5 = pd.read_sql("""
SELECT CT.HOUSEHOLD_KEY AS household_key, COUNT(DISTINCT C.COUPON_UPC) AS CouponSent_Count
FROM CAMPAIGN_TABLE AS CT
LEFT JOIN COUPON AS C
ON CT.CAMPAIGN = C.CAMPAIGN
GROUP BY HOUSEHOLD_KEY""", con, index_col='household_key')

In [16]:
# Creates a column showing the %% of coupons redeemed
df6 = pd.read_sql("""
SELECT CT.HOUSEHOLD_KEY AS household_key, COUNT(DISTINCT R.COUPON_UPC)*1.0/COUNT(DISTINCT C.COUPON_UPC) AS Percent_CouponsRedeemed
FROM CAMPAIGN_TABLE AS CT
LEFT JOIN COUPON AS C
ON CT.CAMPAIGN = C.CAMPAIGN
LEFT JOIN REDEEMED AS R
ON R.HOUSEHOLD_KEY = CT.HOUSEHOLD_KEY
GROUP BY CT.HOUSEHOLD_KEY""", con, index_col='household_key')

In [17]:
# Creates a column showing the total number of days on which a household redeemed a coupon
df7 = pd.read_sql("""
SELECT HOUSEHOLD_KEY, COUNT(DAY) AS Days_coupon_redeemed
FROM REDEEMED
GROUP BY HOUSEHOLD_KEY""", con, index_col='household_key')

In [18]:
# Counts the number of 'Type A' campaigns sent to each household
df8 = pd.read_sql("""
SELECT HOUSEHOLD_KEY, SUM(CASE WHEN DESCRIPTION = 'TypeA' THEN 1 ELSE 0 END) AS TypeA
FROM CAMPAIGN_TABLE
GROUP BY HOUSEHOLD_KEY""", con, index_col='household_key')

In [19]:
# Counts the number of 'Type B' campaigns sent to each household
df9 = pd.read_sql("""
SELECT HOUSEHOLD_KEY, SUM(CASE WHEN DESCRIPTION = 'TypeB' THEN 1 ELSE 0 END) AS TypeB
FROM CAMPAIGN_TABLE
GROUP BY HOUSEHOLD_KEY""", con, index_col='household_key')

In [20]:
# Counts the number of 'Type C' campaigns sent to each household
df10 = pd.read_sql("""
SELECT HOUSEHOLD_KEY, SUM(CASE WHEN DESCRIPTION = 'TypeC' THEN 1 ELSE 0 END) AS TypeC
FROM CAMPAIGN_TABLE
GROUP BY HOUSEHOLD_KEY""", con, index_col='household_key')

In [21]:
# Column for the % of TypeA campaigns sent to each household
df11 = pd.read_sql("""
SELECT HOUSEHOLD_KEY, SUM(CASE WHEN DESCRIPTION = 'TypeA' THEN 1 ELSE 0 END)*1.0/COUNT(DESCRIPTION) AS PercentTypeA
FROM CAMPAIGN_TABLE
GROUP BY HOUSEHOLD_KEY""", con, index_col='household_key')

In [22]:
# Column for the % of TypeA campaigns sent to each household
df12 = pd.read_sql("""
SELECT HOUSEHOLD_KEY, SUM(CASE WHEN DESCRIPTION = 'TypeB' THEN 1 ELSE 0 END)*1.0/COUNT(DESCRIPTION) AS PercentTypeB
FROM CAMPAIGN_TABLE
GROUP BY HOUSEHOLD_KEY""", con, index_col='household_key')

In [23]:
# Column for the % of TypeA campaigns sent to each household
df13 = pd.read_sql("""
SELECT HOUSEHOLD_KEY, SUM(CASE WHEN DESCRIPTION = 'TypeC' THEN 1 ELSE 0 END)*1.0/COUNT(DESCRIPTION) AS PercentTypeC
FROM CAMPAIGN_TABLE
GROUP BY HOUSEHOLD_KEY""", con, index_col='household_key')

In [24]:
# Column for # of unique products purchased by each household
df14 = pd.read_sql("""
SELECT HOUSEHOLD_KEY, COUNT(DISTINCT PRODUCT_ID) AS Count_products
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY""", con, index_col='household_key')

In [25]:
# Creates a column for the # of different deparments households purchased from
df15 = pd.read_sql("""
SELECT T.HOUSEHOLD_KEY AS household_key, COUNT(DISTINCT P.DEPARTMENT) AS Count_depts
FROM TRANSACTION_DATA AS T
LEFT JOIN PRODUCT AS P
ON P.PRODUCT_ID = T.PRODUCT_ID
GROUP BY T.HOUSEHOLD_KEY
""", con, index_col='household_key')

In [26]:
# creates a column with the department a household purchased from the most frequently
df16 = pd.read_sql("""
SELECT Max.HOUSEHOLD_KEY AS household_key, Max.DEPARTMENT AS Most_freq_dept
FROM (SELECT C.HOUSEHOLD_KEY, C.DEPARTMENT, MAX(C.DEPT_COUNT)
FROM (SELECT T.HOUSEHOLD_KEY, P.DEPARTMENT, COUNT(*) AS DEPT_COUNT
FROM TRANSACTION_DATA AS T
LEFT JOIN PRODUCT AS P
ON P.PRODUCT_ID = T.PRODUCT_ID
GROUP BY T.HOUSEHOLD_KEY, P.DEPARTMENT) AS C
GROUP BY C.HOUSEHOLD_KEY) AS Max
""", con, index_col='household_key')

In [27]:
# Creates a column with the department a household purchased from the least frequently
df17 = pd.read_sql("""
SELECT Min.HOUSEHOLD_KEY AS household_key, Min.DEPARTMENT AS Least_freq_dept
FROM (SELECT C.HOUSEHOLD_KEY, C.DEPARTMENT, MIN(C.DEPT_COUNT)
FROM (SELECT T.HOUSEHOLD_KEY, P.DEPARTMENT, COUNT(*) AS DEPT_COUNT
FROM TRANSACTION_DATA AS T
LEFT JOIN PRODUCT AS P
ON P.PRODUCT_ID = T.PRODUCT_ID
GROUP BY T.HOUSEHOLD_KEY, P.DEPARTMENT) AS C
GROUP BY C.HOUSEHOLD_KEY) AS Min
""", con, index_col='household_key')

In [28]:
# Creates a column with the number of unique sub-categories households purchase from
df18 = pd.read_sql("""
SELECT T.HOUSEHOLD_KEY AS household_key, COUNT(DISTINCT P.SUB_COMMODITY_DESC) AS Count_sub_cagtegory
FROM TRANSACTION_DATA AS T
LEFT JOIN PRODUCT AS P
ON T.PRODUCT_ID = P.PRODUCT_ID
GROUP BY HOUSEHOLD_KEY""", con, index_col='household_key')

In [29]:
# Creates a column with the most frequently purchased product
df19 = pd.read_sql("""
SELECT Max.HOUSEHOLD_KEY AS household_key, Max.PRODUCT_ID AS Most_freq_product
FROM (SELECT C.HOUSEHOLD_KEY, C.PRODUCT_ID, MAX(C.Prod_Count)
FROM (SELECT T.HOUSEHOLD_KEY, P.PRODUCT_ID, COUNT(*) AS Prod_Count
FROM TRANSACTION_DATA AS T
LEFT JOIN PRODUCT AS P
ON P.PRODUCT_ID = T.PRODUCT_ID
GROUP BY T.HOUSEHOLD_KEY, P.DEPARTMENT) AS C
GROUP BY C.HOUSEHOLD_KEY) AS Max
""", con, index_col='household_key')

In [30]:
# Creates a column with the least frequently purchased product
df20 = pd.read_sql("""
SELECT Min.HOUSEHOLD_KEY AS household_key, Min.PRODUCT_ID AS Least_freq_product
FROM (SELECT C.HOUSEHOLD_KEY, C.PRODUCT_ID, MIN(C.Prod_Count)
FROM (SELECT T.HOUSEHOLD_KEY, P.PRODUCT_ID, COUNT(*) AS Prod_Count
FROM TRANSACTION_DATA AS T
LEFT JOIN PRODUCT AS P
ON P.PRODUCT_ID = T.PRODUCT_ID
GROUP BY T.HOUSEHOLD_KEY, P.DEPARTMENT) AS C
GROUP BY C.HOUSEHOLD_KEY) AS Min
""", con, index_col='household_key')

In [31]:
# Creates a column showing the product purchased in the largest quantity for each household
df21 = pd.read_sql("""
SELECT Max.HOUSEHOLD_KEY AS household_key, Max.PRODUCT_ID AS High_volumne_product
FROM
(SELECT S.HOUSEHOLD_KEY, PRODUCT_ID, MAX(S.PRODUCT_TOTAL)
FROM
(SELECT HOUSEHOLD_KEY, PRODUCT_ID, SUM(QUANTITY) AS PRODUCT_TOTAL
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, PRODUCT_ID
ORDER BY HOUSEHOLD_KEY, PRODUCT_TOTAL DESC)  AS S
GROUP BY HOUSEHOLD_KEY) AS Max
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [32]:
# Creates a column with the %% of products purchased from a private brand for each household
df23 = pd.read_sql("""
SELECT T.HOUSEHOLD_KEY AS household_key, SUM(CASE WHEN P.BRAND = 'Private' THEN 1 ELSE 0 END)*1.0/COUNT(P.BRAND) AS PercentPrivate
FROM TRANSACTION_DATA AS T
LEFT JOIN PRODUCT AS P
ON T.PRODUCT_ID = P.PRODUCT_ID
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [33]:
# Creates a column with the % of purchased from a households preferred department
df24 = pd.read_sql("""
WITH X AS 
(SELECT Max.HOUSEHOLD_KEY, Max.DEPARTMENT
FROM (SELECT C.HOUSEHOLD_KEY, C.DEPARTMENT, MAX(C.DEPT_COUNT)
FROM (SELECT T.HOUSEHOLD_KEY, P.DEPARTMENT, COUNT(*) AS DEPT_COUNT
FROM TRANSACTION_DATA AS T
LEFT JOIN PRODUCT AS P
ON P.PRODUCT_ID = T.PRODUCT_ID
GROUP BY T.HOUSEHOLD_KEY, P.DEPARTMENT) AS C
GROUP BY C.HOUSEHOLD_KEY) AS Max)

SELECT 

T.HOUSEHOLD_KEY, 
(SUM(CASE WHEN P.DEPARTMENT = X.DEPARTMENT THEN 1 ELSE 0 END)*1.0/COUNT(P.DEPARTMENT))

FROM TRANSACTION_DATA AS T

LEFT JOIN PRODUCT AS P
ON T.PRODUCT_ID = P.PRODUCT_ID

LEFT JOIN X
ON T.HOUSEHOLD_KEY = X.HOUSEHOLD_KEY
GROUP BY T.HOUSEHOLD_KEY
""", con)

In [34]:
# Creates a column with the avgerage basket sales value for each customer
df25 = pd.read_sql("""
SELECT S.HOUSEHOLD_KEY AS household_key, AVG(S.BASKET_TOTAL) AS Avg_basket_sales_value
FROM
(SELECT HOUSEHOLD_KEY, BASKET_ID, SUM(SALES_VALUE) AS BASKET_TOTAL
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, BASKET_ID) AS S
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [35]:
# Creates a column with the maximum basket sales value for each customer
df26 = pd.read_sql("""
SELECT S.HOUSEHOLD_KEY AS household_key, MAX(S.BASKET_TOTAL) AS Max_basket_sales_value
FROM
(SELECT HOUSEHOLD_KEY, BASKET_ID, SUM(SALES_VALUE) AS BASKET_TOTAL
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, BASKET_ID) AS S
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [36]:
# Creates a column with the minimum basket sales value for each customer
df27 = pd.read_sql("""
SELECT S.HOUSEHOLD_KEY AS household_key, MIN(S.BASKET_TOTAL) AS Min_basket_sales_value
FROM
(SELECT HOUSEHOLD_KEY, BASKET_ID, SUM(SALES_VALUE) AS BASKET_TOTAL
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, BASKET_ID) AS S
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [37]:
# Creates a column with the total # of baskets for each household
df28 = pd.read_sql("""
SELECT HOUSEHOLD_KEY AS household_key, COUNT(DISTINCT BASKET_ID) AS Total_num_baskets
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [38]:
# Creates a column with the average number of items in a basket for each household
df29 = pd.read_sql("""
SELECT S.HOUSEHOLD_KEY AS household_key, AVG(S.BASKET_TOTAL) AS Avg_num_items
FROM
(SELECT HOUSEHOLD_KEY, BASKET_ID, SUM(QUANTITY) AS BASKET_TOTAL
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, BASKET_ID) AS S
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [39]:
# Creates a column with the maximum number of items in a basket for each household
df30 = pd.read_sql("""
SELECT S.HOUSEHOLD_KEY AS household_key, MAX(S.BASKET_TOTAL) AS Max_num_items
FROM
(SELECT HOUSEHOLD_KEY, BASKET_ID, SUM(QUANTITY) AS BASKET_TOTAL
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, BASKET_ID) AS S
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [40]:
# Creates a column with the minimum number of items in a basket for each household
df31 = pd.read_sql("""
SELECT S.HOUSEHOLD_KEY AS household_key, MIN(S.BASKET_TOTAL) AS Min_num_items
FROM
(SELECT HOUSEHOLD_KEY, BASKET_ID, SUM(QUANTITY) AS BASKET_TOTAL
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, BASKET_ID) AS S
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [41]:
# Creates a column that shows the most frequent item to appear in each basket
df32 = pd.read_sql("""
SELECT COUNT.HOUSEHOLD_KEY AS household_key, COUNT.PRODUCT_ID AS Most_freq_product, MAX(COUNT.PRODUCT_COUNT)
FROM
(SELECT B.HOUSEHOLD_KEY, B.PRODUCT_ID, COUNT(B.PRODUCT_ID) AS PRODUCT_COUNT
FROM
(SELECT HOUSEHOLD_KEY, BASKET_ID, PRODUCT_ID
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, BASKET_ID) AS B
GROUP BY B.HOUSEHOLD_KEY, B.PRODUCT_ID) AS COUNT
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [42]:
# Creates a column that reflects the average manufacturer's discount
df33 = pd.read_sql("""
SELECT S.HOUSEHOLD_KEY AS household_key, AVG(S.BASKET_TOTAL) AS Avg_manu_disc
FROM
(SELECT HOUSEHOLD_KEY, BASKET_ID, SUM(COUPON_MATCH_DISC) AS BASKET_TOTAL
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, BASKET_ID) AS S
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [43]:
# Creates a column that reflects the maximum manufacturer's discount
df34 = pd.read_sql("""
SELECT S.HOUSEHOLD_KEY AS household_key, MAX(S.BASKET_TOTAL) AS Max_manu_disc
FROM
(SELECT HOUSEHOLD_KEY, BASKET_ID, SUM(COUPON_MATCH_DISC) AS BASKET_TOTAL
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, BASKET_ID) AS S
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [44]:
# Creates a column that reflects the minimum manufacturer's discount
df35 = pd.read_sql("""
SELECT S.HOUSEHOLD_KEY AS household_key, MIN(S.BASKET_TOTAL) AS Min_manu_disc
FROM
(SELECT HOUSEHOLD_KEY, BASKET_ID, SUM(COUPON_MATCH_DISC) AS BASKET_TOTAL
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, BASKET_ID) AS S
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [45]:
# Creates a column that reflects the average coupon discount
df36 = pd.read_sql("""
SELECT S.HOUSEHOLD_KEY AS household_key, AVG(S.BASKET_TOTAL) AS Avg_coupon_disc
FROM
(SELECT HOUSEHOLD_KEY, BASKET_ID, SUM(COUPON_DISC) AS BASKET_TOTAL
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, BASKET_ID) AS S
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [46]:
# Creates a column that reflects the maximum coupon discount
df37 = pd.read_sql("""
SELECT S.HOUSEHOLD_KEY AS household_key, MAX(S.BASKET_TOTAL) AS Max_coupon_disc
FROM
(SELECT HOUSEHOLD_KEY, BASKET_ID, SUM(COUPON_DISC) AS BASKET_TOTAL
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, BASKET_ID) AS S
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [47]:
# Creates a column that reflects the minimum coupon discount
df38 = pd.read_sql("""
SELECT S.HOUSEHOLD_KEY AS household_key, MIN(S.BASKET_TOTAL) AS Min_coupon_disc
FROM
(SELECT HOUSEHOLD_KEY, BASKET_ID, SUM(COUPON_DISC) AS BASKET_TOTAL
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, BASKET_ID) AS S
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [48]:
# Creates a column that reflects the average loyalty discount
df39 = pd.read_sql("""
SELECT S.HOUSEHOLD_KEY AS household_key, AVG(S.BASKET_TOTAL) AS Avg_loyalty_disc
FROM
(SELECT HOUSEHOLD_KEY, BASKET_ID, SUM(RETAIL_DISC) AS BASKET_TOTAL
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, BASKET_ID) AS S
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [49]:
# Creates a column that reflects the maximum loyalty discount
df40 = pd.read_sql("""
SELECT S.HOUSEHOLD_KEY AS household_key, MAX(S.BASKET_TOTAL) AS Max_loyalty_disc
FROM
(SELECT HOUSEHOLD_KEY, BASKET_ID, SUM(RETAIL_DISC) AS BASKET_TOTAL
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, BASKET_ID) AS S
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [50]:
# Creates a column that reflects the minimum loyalty discount
df41 = pd.read_sql("""
SELECT S.HOUSEHOLD_KEY AS household_key, MIN(S.BASKET_TOTAL) AS Min_loyalty_disc
FROM
(SELECT HOUSEHOLD_KEY, BASKET_ID, SUM(RETAIL_DISC) AS BASKET_TOTAL
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, BASKET_ID) AS S
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [51]:
# Creates a column that reflects the # of stores purchased from
df42 = pd.read_sql("""
SELECT HOUSEHOLD_KEY AS household_key, COUNT(DISTINCT STORE_ID) AS Num_stores_visited
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [52]:
# Creates a column showing the most frequent store purchased from
df43 = pd.read_sql("""
SELECT Max.HOUSEHOLD_KEY AS household_key, Max.STORE_ID AS Most_freq_store
FROM (SELECT C.HOUSEHOLD_KEY, C.STORE_ID, MAX(C.Prod_Count)
FROM (SELECT HOUSEHOLD_KEY, STORE_ID, COUNT(*) AS Prod_Count
FROM TRANSACTION_DATA AS T
GROUP BY HOUSEHOLD_KEY, STORE_ID) AS C
GROUP BY C.HOUSEHOLD_KEY) AS Max
""", con, index_col='household_key')

In [53]:
# Creates a column reflecting the most frequent time of day to purchase
df44 = pd.read_sql("""
SELECT Max.HOUSEHOLD_KEY AS household_key, Max.TRANS_TIME AS Most_freq_time
FROM (SELECT C.HOUSEHOLD_KEY, C.TRANS_TIME, MAX(C.Prod_Count)
FROM (SELECT HOUSEHOLD_KEY, TRANS_TIME, COUNT(*) AS Prod_Count
FROM TRANSACTION_DATA AS T
GROUP BY HOUSEHOLD_KEY, STORE_ID) AS C
GROUP BY C.HOUSEHOLD_KEY) AS Max
""", con, index_col='household_key')

In [54]:
# Creates a column reflecting the 1st day a household was active
df45 = pd.read_sql("""
SELECT HOUSEHOLD_KEY, MIN(DAY) AS First_active_day
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [55]:
# Creates a column reflecting the last day of purchase for a household
df46 = pd.read_sql("""
SELECT HOUSEHOLD_KEY, MAX(DAY) AS Last_active_day
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY
""",con, index_col='household_key')

In [56]:
# Creates a column reflecting the Recency of a household
df47 = pd.read_sql("""
SELECT HOUSEHOLD_KEY, (711-MAX(DAY)) AS Recency
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [57]:
# Creates a column reflecting the Frequency of a household
df48 = pd.read_sql("""

SELECT HOUSEHOLD_KEY, (COUNT(DISTINCT BASKET_ID)*1.0/711) AS Frequency
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [58]:
# Creates a column reflecting the Monetary value of a household
df49 = pd.read_sql("""
SELECT HOUSEHOLD_KEY, SUM(SALES_VALUE) AS Monetary
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [59]:
# Creates a column reflecting the average # of trips per week
df50 = pd.read_sql("""
SELECT B.HOUSEHOLD_KEY AS household_key, AVG(B.TRIP_COUNT) AS Avg_trips_week
FROM
(SELECT HOUSEHOLD_KEY, WEEK_NO, COUNT(DISTINCT BASKET_ID) AS TRIP_COUNT
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, WEEK_NO) AS B
GROUP BY B.HOUSEHOLD_KEY
""", con, index_col='household_key')

In [60]:
# Creates a column reflecting the %% of purchases made where the product purchased was displayed in the store
df51 = pd.read_sql('''
WITH X AS
(SELECT T.HOUSEHOLD_KEY, COUNT(T.PRODUCT_ID) AS A
FROM TRANSACTION_DATA AS T
LEFT JOIN DISPLAY_DATA AS D
ON T.PRODUCT_ID = D.PRODUCT_ID
WHERE D.WEEK_NO = T.WEEK_NO AND D.STORE_ID=T.STORE_ID AND D.PRODUCT_ID=T.PRODUCT_ID AND DISPLAY <> '0'
GROUP BY T.HOUSEHOLD_KEY)

SELECT T.HOUSEHOLD_KEY AS household_key, X.A*1.0/COUNT(T.PRODUCT_ID)
FROM TRANSACTION_DATA AS T
LEFT JOIN X
ON X.HOUSEHOLD_KEY = T.HOUSEHOLD_KEY
GROUP BY T.HOUSEHOLD_KEY''', con, index_col = 'household_key')

In [61]:
# Creates a column reflecting the %% of purchases made where the product purchased was displayed in the mailer the
# week of or week prior to the purchase
df52 = pd.read_sql("""
WITH X AS
(SELECT T.HOUSEHOLD_KEY, COUNT(T.PRODUCT_ID) AS A
FROM TRANSACTION_DATA AS T
LEFT JOIN DISPLAY_DATA AS D
ON T.PRODUCT_ID = D.PRODUCT_ID
WHERE D.WEEK_NO = T.WEEK_NO AND D.STORE_ID=T.STORE_ID AND D.PRODUCT_ID=T.PRODUCT_ID AND MAILER <> '0'
GROUP BY T.HOUSEHOLD_KEY)

SELECT T.HOUSEHOLD_KEY AS household_key, X.A*1.0/COUNT(T.PRODUCT_ID)
FROM TRANSACTION_DATA AS T
LEFT JOIN X
ON X.HOUSEHOLD_KEY = T.HOUSEHOLD_KEY
GROUP BY T.HOUSEHOLD_KEY
""", con, index_col='household_key')

In [62]:
# Creates a column that shows what %% of baskets included an item which was on display in the mailer
df53 = pd.read_sql("""
WITH X AS
(SELECT T.HOUSEHOLD_KEY, COUNT(T.BASKET_ID) AS A
FROM TRANSACTION_DATA AS T
LEFT JOIN DISPLAY_DATA AS D
ON T.PRODUCT_ID = D.PRODUCT_ID
WHERE D.WEEK_NO = T.WEEK_NO AND D.STORE_ID=T.STORE_ID AND D.PRODUCT_ID=T.PRODUCT_ID AND MAILER <> '0'
GROUP BY T.HOUSEHOLD_KEY)

SELECT T.HOUSEHOLD_KEY AS household_key, X.A*1.0/COUNT(T.BASKET_ID)
FROM TRANSACTION_DATA AS T
LEFT JOIN X
ON X.HOUSEHOLD_KEY = T.HOUSEHOLD_KEY
GROUP BY T.HOUSEHOLD_KEY
""", con, index_col='household_key')

In [63]:
# Creates a column which shows the %% of baskets which included an item on display in the store
df54 = pd.read_sql("""
WITH X AS
(SELECT T.HOUSEHOLD_KEY, COUNT(T.BASKET_ID) AS A
FROM TRANSACTION_DATA AS T
LEFT JOIN DISPLAY_DATA AS D
ON T.PRODUCT_ID = D.PRODUCT_ID
WHERE D.WEEK_NO = T.WEEK_NO AND D.STORE_ID=T.STORE_ID AND D.PRODUCT_ID=T.PRODUCT_ID AND DISPLAY <> '0'
GROUP BY T.HOUSEHOLD_KEY)

SELECT T.HOUSEHOLD_KEY AS household_key, X.A*1.0/COUNT(T.BASKET_ID)
FROM TRANSACTION_DATA AS T
LEFT JOIN X
ON X.HOUSEHOLD_KEY = T.HOUSEHOLD_KEY
GROUP BY T.HOUSEHOLD_KEY
""", con, index_col = 'household_key')

In [64]:
# Creates a column for the average quantity of items purchased in a given trip (eg 'bulk shoppers')
df55 = pd.read_sql("""
SELECT HOUSEHOLD_KEY, AVG(QUANTITY)
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY
""", con, index_col='household_key')

In [65]:
# Creates a column that reflects the average number of days between shopping instances
df56 = pd.read_sql("""WITH X AS
(SELECT HOUSEHOLD_KEY, BASKET_ID, DAY
FROM TRANSACTION_DATA
GROUP BY HOUSEHOLD_KEY, BASKET_ID
ORDER BY HOUSEHOLD_KEY, DAY),

Y AS
(SELECT *, LAG(DAY, 1) OVER(Partition by HOUSEHOLD_KEY order by DAY) AS Z
FROM X),

W AS
(SELECT *, (DAY - Z) AS DELTA
FROM Y)

SELECT HOUSEHOLD_KEY as household_key, AVG(DELTA) AS Avg_shopping_lag
FROM W
WHERE DELTA IS NOT NULL
GROUP BY HOUSEHOLD_KEY""", con, index_col='household_key')

In [66]:
df57 = pd.read_sql("""SELECT * FROM HH_DEMOGRAPHIC""", con, index_col='household_key')
df57

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC
household_key,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
7,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown
8,25-34,U,25-34K,Unknown,2 Adults Kids,3,1
13,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2
16,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown
...,...,...,...,...,...,...,...
2494,35-44,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown
2496,45-54,A,75-99K,Homeowner,Unknown,3,1
2497,45-54,U,35-49K,Unknown,Single Male,1,None/Unknown


In [67]:
DF = pd.concat([df57, df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18, 
                df19, df20, df21, df23, df24, df51, df52, df53, df54, df55,  df25, df26, df27, df28, df29, df30, df31, df32, df33, df34, df35, df36, df37, 
                df38, df39, df40, df41, df42, df43, df44, df45, df46, df47, df48, df49, df50, df56], axis=1)
DF.fillna(0, inplace=True)   # Filling non-values where households didn't interact with any campaigns with zero
DF

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,CampaignsRedeemed,CampaignsSent,Percent_CampaignRedeemed,...,Num_stores_visited,Most_freq_store,Most_freq_time,First_active_day,Last_active_day,Recency,Frequency,Monetary,Avg_trips_week,Avg_shopping_lag
0,0,0,0,0,0,0,0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,2.0,8.0,0.25,...,2.0,436.0,1456.0,51.0,706.0,5.0,0.120956,4330.16,1.264706,7.705882
2,0,0,0,0,0,0,0,0.0,1.0,0.00,...,5.0,401.0,1904.0,103.0,668.0,43.0,0.063291,1954.34,1.285714,12.840909
3,0,0,0,0,0,0,0,0.0,3.0,0.00,...,3.0,401.0,1549.0,113.0,703.0,8.0,0.066104,2653.21,1.270270,12.826087
4,0,0,0,0,0,0,0,0.0,1.0,0.00,...,6.0,298.0,1452.0,104.0,627.0,84.0,0.042194,1200.11,1.153846,18.034483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,45-54,A,75-99K,Homeowner,Unknown,3,1,3.0,5.0,0.60,...,3.0,370.0,1345.0,117.0,683.0,28.0,0.088608,4339.66,1.312500,9.129032
2497,45-54,U,35-49K,Unknown,Single Male,1,None/Unknown,0.0,3.0,0.00,...,12.0,31742.0,1001.0,78.0,700.0,11.0,0.310830,7111.98,2.569767,2.827273
2498,25-34,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,0.0,6.0,0.00,...,7.0,309.0,1823.0,105.0,710.0,1.0,0.241913,2601.60,2.567164,3.538012
2499,25-34,U,Under 15K,Unknown,2 Adults Kids,3,1,0.0,2.0,0.00,...,9.0,447.0,1923.0,70.0,709.0,2.0,0.126582,3394.07,1.666667,7.179775


In [68]:
DF.to_csv('../Data/Aggregate_Customer_Data')

At this point I've aggregated information on each of the 2500 households in this data set for clustering analysis and segmentation.  

Moving forward I'll need to do some initial data wrangling and exploratory data analysis prior to settling on an approach to use with clustering algorithms.  These exploratory and preperatory steps will be conducted in the subsequent notebook in this repository. 